In [ ]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta, timezone
import os

# --- 1. XMLデータの取得 ---
# 気象防災情報XMLの公開ページURLを定義します。
KISHOU_XML_PAGE_URL = "https://www.data.jma.go.jp/developer/xml/feed/extra_l.xml"

# メインフィードXMLデータと、各エントリーへのリンクされたXMLデータを格納するリスト
fetched_xml_data = {
    "main_feed_xml": None,
    "linked_entries_xml": []
}

# 時間の閾値（現在から48時間前）を定義します。
time_threshold = datetime.now(timezone.utc) - timedelta(hours=48)
print(f"以下の時間より前に更新されたエントリーはフィルタリングされます: {time_threshold.isoformat()}")

try:
    print(f"メインAtomフィードを以下から取得します: {KISHOU_XML_PAGE_URL}")
    main_xml_response = requests.get(KISHOU_XML_PAGE_URL)
    main_xml_response.raise_for_status()
    fetched_xml_data["main_feed_xml"] = main_xml_response.content
    print("メインAtomフィードXMLデータを正常に取得しました。")

    if fetched_xml_data["main_feed_xml"]:
        main_feed_root = ET.fromstring(fetched_xml_data["main_feed_xml"].decode('utf-8'))
        atom_ns = '{http://www.w3.org/2005/Atom}'

        for entry in main_feed_root.findall(f'{atom_ns}entry'):
            entry_info = {}
            entry_info['EntryID'] = entry.find(f'{atom_ns}id').text if entry.find(f'{atom_ns}id') is not None else "N/A"
            entry_info['FeedReportDateTime'] = entry.find(f'{atom_ns}updated').text if entry.find(f'{atom_ns}updated') is not None else "N/A"
            entry_info['FeedTitle'] = entry.find(f'{atom_ns}title').text if entry.find(f'{atom_ns}title') is not None else "N/A"
            entry_info['Author'] = entry.find(f'{atom_ns}author/{atom_ns}name').text if entry.find(f'{atom_ns}author/{atom_ns}name') is not None else "N/A"

            feed_report_time_str = entry_info.get('FeedReportDateTime')
            if feed_report_time_str and feed_report_time_str != 'N/A':
                try:
                    if feed_report_time_str.endswith('Z'):
                        feed_report_time = datetime.fromisoformat(feed_report_time_str[:-1]).replace(tzinfo=timezone.utc)
                    else:
                        feed_report_time = datetime.fromisoformat(feed_report_time_str)

                    if feed_report_time < time_threshold:
                        continue

                except ValueError as e:
                     print(f"警告: エントリー {entry_info.get('EntryID', 'N/A')} のタイムスタンプ '{feed_report_time_str}' を解析できませんでした: {e}。リンクされたXMLの取得を試行します。")
                except Exception as e:
                     print(f"エントリー {entry_info.get('EntryID', 'N/A')} のタイムスタンプ処理中に予期しないエラーが発生しました: {e}。リンクされたXMLの取得を試行します。")

            linked_xml_link_element = entry.find(f'{atom_ns}link[@type="application/xml"]')

            if linked_xml_link_element is not None:
                linked_xml_url = linked_xml_link_element.get('href')
                if linked_xml_url:
                    try:
                        linked_xml_response = requests.get(linked_xml_url)
                        linked_xml_response.raise_for_status()
                        entry_info['LinkedXMLData'] = linked_xml_response.content
                    except requests.exceptions.RequestException as e:
                        print(f"'{linked_xml_url}' からリンクされたXMLを取得中にエラーが発生しました: {e}")
                        entry_info['LinkedXMLData'] = None
                    except Exception as e:
                         print(f"'{linked_xml_url}' からリンクされたXMLを取得中に予期しないエラーが発生しました: {e}")
                         entry_info['LinkedXMLData'] = None
                else:
                    entry_info['LinkedXMLData'] = None
            else:
                entry_info['LinkedXMLData'] = None

            fetched_xml_data["linked_entries_xml"].append(entry_info)

        print(f"メインフィードから {len(fetched_xml_data['linked_entries_xml'])} 件のエントリーを処理しました。")

    else:
        print("メインAtomフィードXMLデータは取得されませんでした。")


except requests.exceptions.RequestException as e:
    print(f"{KISHOU_XML_PAGE_URL} への初回リクエスト中にエラーが発生しました: {e}")
except Exception as e:
    print(f"予期しないエラーが発生しました: {e}")


# --- 2. XMLデータの解析 ---
parsed_linked_data = []

if fetched_xml_data and fetched_xml_data.get("linked_entries_xml"):
    print("\n各エントリーのリンクされたXMLデータを解析しています...")
    for entry_data in fetched_xml_data["linked_entries_xml"]:
        linked_xml_bytes = entry_data.get('LinkedXMLData')

        extracted_entry = {
            "EntryID": entry_data.get('EntryID', 'N/A'),
            "FeedReportDateTime": entry_data.get('FeedReportDateTime', 'N/A'),
            "FeedTitle": entry_data.get('FeedTitle', 'N/A'),
            "Author": entry_data.get('Author', 'N/A'),
            "LinkedXMLDataPresent": linked_xml_bytes is not None
        }

        feed_report_time_str = extracted_entry.get('FeedReportDateTime')
        if feed_report_time_str and feed_report_time_str != 'N/A':
            try:
                if feed_report_time_str.endswith('Z'):
                    feed_report_time = datetime.fromisoformat(feed_report_time_str[:-1]).replace(tzinfo=timezone.utc)
                else:
                    feed_report_time = datetime.fromisoformat(feed_report_time_str)

                if feed_report_time < time_threshold:
                    continue

            except ValueError as e:
                print(f"警告: エントリー {extracted_entry.get('EntryID', 'N/A')} のタイムスタンプ '{feed_report_time_str}' を解析できませんでした: {e}。このエントリーのタイムスタンプフィルタをスキップします。")
            except Exception as e:
                 print(f"エントリー {extracted_entry.get('EntryID', 'N/A')} のタイムスタンプ処理中に予期しないエラーが発生しました: {e}。このエントリーのタイムスタンプフィルタをスキップします。")

        if extracted_entry.get('FeedTitle') != "気象特別警報・警報・注意報":
            continue

        warnings_advisories_info = []

        if linked_xml_bytes:
            try:
                linked_xml_string = linked_xml_bytes.decode('utf-8')
                linked_root = ET.fromstring(linked_xml_string)

                report_time_element = linked_root.find('.//{*}ReportDateTime')
                extracted_entry['ReportDateTime'] = report_time_element.text if report_time_element is not None else extracted_entry['FeedReportDateTime']

                headline_text_element = linked_root.find('.//{*}Headline/{*}Text')
                overall_detail_text = headline_text_element.text if headline_text_element is not None else "N/A"

                item_elements = linked_root.findall('.//{*}Item')
                if not item_elements:
                    item_elements = linked_root.findall('.//{*}Headline//{*}Item')

                for item in item_elements:
                    kind = "N/A"
                    area = "N/A"
                    area_code = "N/A"
                    detail = overall_detail_text

                    kind_element = item.find('.//{*}Kind/{*}Name')
                    kind = kind_element.text if kind_element is not None else "N/A"

                    area_element = item.find('.//{*}Areas/{*}Area/{*}Name')
                    if area_element is not None:
                         area = area_element.text
                         area_code_element = item.find('.//{*}Areas/{*}Area/{*}Code')
                         area_code = area_code_element.text if area_code_element is not None else "N/A"
                    else:
                         area_element = item.find('.//{*}Areas/{*}Area/{*}Prefecture/{*}Name')
                         if area_element is not None:
                              area = area_element.text
                              area_code_element = item.find('.//{*}Areas/{*}Area/{*}Prefecture/{*}Code')
                              area_code = area_code_element.text if area_code_element is not None else "N/A"
                         else:
                              area_element = item.find('.//{*}Area/{*}Name')
                              if area_element is not None:
                                   area = area_element.text
                                   area_code_element = item.find('.//{*}Area/{*}Code')
                                   area_code = area_code_element.text if area_code_element is not None else "N/A"
                              else:
                                   area_code_element = item.find('.//{*}Area/{*}Code')
                                   if area_code_element is not None:
                                        area_code = area_code_element.text
                                        area = f"コード:{area_code}"
                                   else:
                                       area = "N/A"
                                       area_code = "N/A"

                    if kind != "N/A" or area != "N/A" or area_code != "N/A":
                         warnings_advisories_info.append({"Kind": kind, "Area": area, "AreaCode": area_code, "Detail": detail})
                    elif item is not None:
                          warnings_advisories_info.append({"Kind": "不明な種類", "Area": "不明な地域", "AreaCode": "N/A", "Detail": detail})

            except ET.ParseError as e:
                print(f"エントリー {extracted_entry.get('EntryID', 'N/A')} のリンクされたXMLの解析エラー: {e}")
                extracted_entry['ReportDateTime'] = extracted_entry.get('FeedReportDateTime', '解析エラー')
                extracted_entry['WarningsAdvisories'] = [{"Kind": "解析エラー", "Area": "解析エラー", "AreaCode": "解析エラー", "Detail": "解析エラー"}]
            except Exception as e:
                print(f"エントリー {extracted_entry.get('EntryID', 'N/A')} のリンクされたXML解析中に予期しないエラーが発生しました: {e}")
                extracted_entry['ReportDateTime'] = extracted_entry.get('FeedReportDateTime', 'エラー')
                extracted_entry['WarningsAdvisories'] = [{"Kind": "エラー", "Area": "エラー", "AreaCode": "エラー", "Detail": "エラー"}]

        else:
            extracted_entry['ReportDateTime'] = extracted_entry.get('FeedReportDateTime', '取得失敗')
            extracted_entry['WarningsAdvisories'] = [{"Kind": "取得失敗", "Area": "取得失敗", "AreaCode": "取得失敗", "Detail": "取得失敗"}]

        if warnings_advisories_info:
             extracted_entry['WarningsAdvisories'] = warnings_advisories_info
             parsed_linked_data.append(extracted_entry)


    print(f"'気象特別警報・警報・注意報'のリンクされたXMLファイル {len(parsed_linked_data)} 件からデータを解析しました。")

else:
    print("解析するリンクされたエントリーXMLデータがありません。")
    parsed_linked_data = []


# --- 3. 地域コードデータの読み込みと準備 ---
excel_file = '/content/100323AreaInfomationCity-AreaForecastLocalM.xls'
area_code_name_mapping = {}

try:
    area_code_df_raw = pd.read_excel(excel_file, header=None, engine='xlrd')

    code_col_index = None
    name_col_index = None

    first_data_row = area_code_df_raw.iloc[2]

    for i, value in enumerate(first_data_row):
        if isinstance(value, str):
            if value.strip() == '@code':
                code_col_index = i
            elif value.strip() == '@name':
                name_col_index = i

    if code_col_index is not None and name_col_index is not None:
        area_mapping_df = area_code_df_raw.iloc[3:, [code_col_index, name_col_index]].copy()
        area_mapping_df.columns = ['@code', '@name']
        area_mapping_df['@code'] = area_mapping_df['@code'].astype(str)
        area_code_name_mapping = area_mapping_df.set_index('@code')['@name'].dropna().to_dict()
        print(f"地域コードマッピング辞書が利用可能です。{len(area_code_name_mapping)}件のエントリがあります。")

    else:
        print("エラー: 必要な値('@code'または'@name')が最初のデータ行に見つかりません。ファイルの内容を確認してください。")

except FileNotFoundError:
    print(f"エラー: ファイルが見つかりません: {excel_file}")
except ImportError:
    print("エラー: 'xlrd' ライブラリが必要です。インストールするには `%pip install xlrd` を実行してください。")
except Exception as e:
    print(f"'{excel_file}' の読み込みまたは処理中にエラーが発生しました: {e}")


# --- 4. データ構造の変換と列操作 ---
transformed_data_for_db = []

if parsed_linked_data:
    print(f"\nデータベース用に {len(parsed_linked_data)} 件の解析済みエントリーを変換しています...")
    for entry_data in parsed_linked_data:
        entry_id = entry_data.get('EntryID', 'N/A')
        report_datetime = entry_data.get('ReportDateTime', entry_data.get('FeedReportDateTime', 'N/A'))
        title = entry_data.get('FeedTitle', 'N/A')
        author = entry_data.get('Author', 'N/A')
        warnings_advisories = entry_data.get('WarningsAdvisories', [])

        if warnings_advisories:
            for warning_info in warnings_advisories:
                area = warning_info.get('Area', 'N/A')
                area_code = warning_info.get('AreaCode', 'N/A')

                transformed_data_for_db.append({
                    "ReportDateTime": report_datetime,
                    "Title": title,
                    "Author": author,
                    "AreaCode": area_code,
                    "Area": area,
                    "Kind": warning_info.get('Kind', 'N/A'),
                    "Detail": warning_info.get('Detail', 'N/A'),
                    "EntryID": entry_id
                })
        else:
            transformed_data_for_db.append({
                 "ReportDateTime": report_datetime,
                 "Title": title,
                 "Author": author,
                 "AreaCode": "N/A",
                 "Area": "詳細な警告は見つかりません",
                 "Kind": "N/A",
                 "Detail": "N/A",
                 "EntryID": entry_id
            })

    transformed_data_df = pd.DataFrame(transformed_data_for_db)

    transformed_data_df = transformed_data_df.rename(columns={
        "Area": "気象情報／府県予報区・細分区域等",
        "AreaCode": "「AreaForecastLocalM」コード"
    })

    target_columns_order = [
        "ReportDateTime",
        "Title",
        "Author",
        "「AreaForecastLocalM」コード",
        "気象情報／府県予報区・細分区域等",
        "Kind",
        "Detail",
        "EntryID"
    ]
    ordered_columns = [col for col in target_columns_order if col in transformed_data_df.columns]
    transformed_data_df = transformed_data_df[ordered_columns]

    print("\nデータベース用の変換データ（DataFrame、列名変更＆順序調整後）:")
    display(transformed_data_df.head())

else:
    print("変換する解析済みのリンクされたデータがありません。空のDataFrameを作成します。")
    transformed_data_df = pd.DataFrame(columns=["ReportDateTime", "Title", "Author", "「AreaForecastLocalM」コード", "気象情報／府県予報区・細分区域等", "Kind", "Detail", "EntryID"])
    display(transformed_data_df.head())


# --- 5. ピボットテーブルの再作成と保存 ---
if 'transformed_data_df' in locals() and transformed_data_df is not None and not transformed_data_df.empty:
    print("\n更新されたデータフレームを使用して手動でピボットテーブルを作成しています...")

    try:
        transformed_data_df['ReportDateTime_cleaned'] = pd.to_datetime(transformed_data_df['ReportDateTime'], errors='coerce')
        transformed_data_df['ReportDateTime_cleaned'] = transformed_data_df['ReportDateTime_cleaned'].dt.tz_convert(None)
        transformed_data_df['ReportDate'] = transformed_data_df['ReportDateTime_cleaned'].dt.date
        report_date_col_exists = True
    except Exception as e:
        print(f"ReportDateTimeの変換エラー: {e}")
        report_date_col_exists = False

    manual_pivot_index_cols = ['ReportDate', 'Title', 'Author', '「AreaForecastLocalM」コード', '気象情報／府県予報区・細分区域等']
    if not report_date_col_exists:
        manual_pivot_index_cols.remove('ReportDate')

    try:
        # Ensure 'Kind' column is string type before manual pivot
        transformed_data_df['Kind'] = transformed_data_df['Kind'].astype(str)

        manual_pivot_df = transformed_data_df.groupby(manual_pivot_index_cols + ['Kind']).size().unstack(fill_value=0)

        print("\n各地域ごとの警報/注意報の発令状況（手動で作成されたピボットテーブル）:")
        display(manual_pivot_df.head())

    except Exception as e:
        print(f"手動ピボットテーブル作成エラー: {e}")
        manual_pivot_df = pd.DataFrame()

    output_pivot_csv_file = 'kishou_warnings_advisories_pivot.csv'

    try:
        manual_pivot_df.to_csv(output_pivot_csv_file, encoding='utf-8-sig')
        print(f"手動で作成されたピボットテーブルが'{output_pivot_csv_file}'に正常に保存されました。")
    except Exception as e:
        print(f"手動ピボットテーブルのCSV保存エラー: {e}")

else:
    print("\n処理するための変換されたデータがありません。ピボットテーブルを作成できません。")
    manual_pivot_df = pd.DataFrame()

以下の時間より前に更新されたエントリーはフィルタリングされます: 2025-10-24T12:07:15.585791+00:00
メインAtomフィードを以下から取得します: https://www.data.jma.go.jp/developer/xml/feed/extra_l.xml
メインAtomフィードXMLデータを正常に取得しました。
メインフィードから 504 件のエントリーを処理しました。

各エントリーのリンクされたXMLデータを解析しています...
'気象特別警報・警報・注意報'のリンクされたXMLファイル 230 件からデータを解析しました。
エラー: ファイルが見つかりません: /content/100323AreaInfomationCity-AreaForecastLocalM.xls

データベース用に 230 件の解析済みエントリーを変換しています...

データベース用の変換データ（DataFrame、列名変更＆順序調整後）:


,ReportDateTime,Title,Author,「AreaForecastLocalM」コード,気象情報／府県予報区・細分区域等,Kind,Detail,EntryID
0,2025-10-26T20:50:00+09:00,気象特別警報・警報・注意報,長野地方気象台,200000,長野県,大雨注意報,北部では、２６日夜遅くまで土砂災害や河川の増水に注意してください。中部では２７日昼前まで、北...,https://www.data.jma.go.jp/developer/xml/data/...
1,2025-10-26T20:50:00+09:00,気象特別警報・警報・注意報,長野地方気象台,200010,北部,大雨注意報,北部では、２６日夜遅くまで土砂災害や河川の増水に注意してください。中部では２７日昼前まで、北...,https://www.data.jma.go.jp/developer/xml/data/...
2,2025-10-26T20:50:00+09:00,気象特別警報・警報・注意報,長野地方気象台,200020,中部,濃霧注意報,北部では、２６日夜遅くまで土砂災害や河川の増水に注意してください。中部では２７日昼前まで、北...,https://www.data.jma.go.jp/developer/xml/data/...
3,2025-10-26T20:50:00+09:00,気象特別警報・警報・注意報,長野地方気象台,200030,南部,濃霧注意報,北部では、２６日夜遅くまで土砂災害や河川の増水に注意してください。中部では２７日昼前まで、北...,https://www.data.jma.go.jp/developer/xml/data/...
4,2025-10-26T20:50:00+09:00,気象特別警報・警報・注意報,長野地方気象台,200011,長野地域,大雨注意報,北部では、２６日夜遅くまで土砂災害や河川の増水に注意してください。中部では２７日昼前まで、北...,https://www.data.jma.go.jp/developer/xml/data/...



更新されたデータフレームを使用して手動でピボットテーブルを作成しています...

各地域ごとの警報/注意報の発令状況（手動で作成されたピボットテーブル）:


Kind                                                                       N/A  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等        
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                 0   
                                 350010                  西部                  0   
                                 350011                  下関                  0   
                                 350012                  宇部・山陽小野田            0   
                                 350020                  中部                  0   

Kind                                                                       乾燥注意報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等          
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                   0   
                                 350010                  西部                    0   
                                 350011                  下関                    0   
                                 350012                  宇部・山陽小野田              0   
                                 350020                  中部                    0   

Kind                                                                       低温注意報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等          
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                   0   
                                 350010                  西部                    0   
                                 350011                  下関                    0   
                                 350012                  宇部・山陽小野田              0   
                                 350020                  中部                    0   

Kind                                                                       大雨注意報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等          
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                   0   
                                 350010                  西部                    0   
                                 350011                  下関                    0   
                                 350012                  宇部・山陽小野田              0   
                                 350020                  中部                    0   

Kind                                                                       大雨警報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等         
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                  0   
                                 350010                  西部                   0   
                                 350011                  下関                   0   
                                 350012                  宇部・山陽小野田             0   
                                 350020                  中部                   0   

Kind                                                                       強風注意報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等          
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                   5   
                                 350010                  西部                    3   
                                 350011                  下関                    3   
                                 350012                  宇部・山陽小野田              3   
                                 350020                  中部                    3   

Kind                                                                       波浪注意報  \
ReportDate Title         Author  「AreaForecastLocalM」コード 気象情報／府県予報区・細分区域等          
2025-10-24 気象特別警報・警報・注意報 下関地方気象台 350000                  山口県                   0   
                                 350010                  西部                    0   
                                 350011                  下関                    0   
                                 350012                  宇部・山陽小野田  

手動で作成されたピボットテーブルが'kishou_warnings_advisories_pivot.csv'に正常に保存されました。
